## The training data contains profile images collected from themovedb.org and labeled for each celebrity in the top 100 list.
* https://www.themoviedb.org/person/6384-keanu-reeves/images/profiles?language=en-US

* Extract the facial features and labels from the training data set
* Save the vectors into pickle files for the classifier

### Run the Docker image to install all the necessary libraries for the image and facial processing.


In [1]:
# Configure Environment 
from  prod.data.settings import * 
print("Platform: " , PLATFORM)
print("Data directory : ", os.getcwd())




Platform:  dev
Data directory :  /home/jovyan/dev/data


In [2]:
import math
from sklearn import neighbors
import os
import os.path
from pprint import pprint
import pickle
import pandas as pd
from PIL import Image, ImageDraw

ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}
import logging
logger = logging.getLogger('facialRecognition')
logger.setLevel(logging.DEBUG)


## Process celebrity images and perform facial recognition using the Dlib and OpenFace AI libraries.
* Crawl through the celebrity image folders.  Each folder is labeled with the actorID
* Store the facial vectors and labels for the classifcation models.


In [3]:
!pip install face_recognition

You are using pip version 9.0.3, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [5]:
import face_recognition
from face_recognition.face_recognition_cli import image_files_in_folder

X = []
y = []
images = []

train_dir="./person/images"

verbose=True
# Loop through each person in the training set
for class_dir in os.listdir(train_dir):
    if not os.path.isdir(os.path.join(train_dir, class_dir)):
        continue

    # Loop through each training image for the current person
    for img_path in image_files_in_folder(os.path.join(train_dir, class_dir)):
        logger.warning('Process image : '+img_path)
        try:
            image = face_recognition.load_image_file(img_path)
            face_bounding_boxes = face_recognition.face_locations(image)

            if len(face_bounding_boxes) != 1:
                # If there are no people (or too many people) in a training image, skip the image.
                if verbose:
                    print("Image {} not suitable for training: {}".format(img_path, "Didn't find a face" if len(face_bounding_boxes) < 1 else "Found more than one face"))
            else:
                # Add face encoding for current image to the training set
                X.append(face_recognition.face_encodings(image, known_face_locations=face_bounding_boxes)[0])
                y.append(class_dir)
                images.append(img_path)
        except Exception as e:
            logger.error('Failed to retrieve data : '+img_path+' :: '+ str(e))




Process image : ./person/images/1/themoviedb_i_1_5A7vGrVJcOLdfow1i9GoXN85Q16.jpg
Process image : ./person/images/1/themoviedb_i_1_qI5nBHRFhaPO3WK5YJ8bCvqW1Jv.jpg
Process image : ./person/images/1/themoviedb_i_1_1YjdSym1jTG7xjHSI0yGGWEsw5i.jpg
Process image : ./person/images/1/themoviedb_i_1_iDtxXJXmJGU7jYRjsvePNQblHV1.jpg
Process image : ./person/images/1/themoviedb_i_1_dRLSoufWtc16F5fliK4ECIVs56p.jpg
Process image : ./person/images/1/themoviedb_i_1_5qHNjhtjMD4YWH3UP0rm4tKwxCL.jpg
Process image : ./person/images/1/themoviedb_i_1_zY5osf94NzImi2FoqSQvdSb0SQz.jpg
Process image : ./person/images/1/themoviedb_i_1_r7WLn4Kbnqb6oJ8TmSI0e4LkWTj.jpg
Process image : ./person/images/1/themoviedb_i_1_8fYa7iCeaPzkeTlmuZdBU1be8q5.jpg
Process image : ./person/images/10/themoviedb_i_10_4p1AqbCePPXnn2H00Mv4LavVvCK.jpg
Process image : ./person/images/10/themoviedb_i_10_7CcoVFTogQgex2kJkXKMe8qHZrC.jpg
Process image : ./person/images/10/themoviedb_i_10_5Vj23wvkRVrx2Lc2lVZh5MEbIwg.jpg
Process image : ./pers

Image ./person/images/11/themoviedb_i_11_lbUAkDqKTRpNVpXVM3eAQbqXLzS.jpg not suitable for training: Didn't find a face


Process image : ./person/images/11/themoviedb_i_11_fL5MURVXzWK8JCDjE8bPfwYgJcp.jpg
Process image : ./person/images/11/themoviedb_i_11_lfMylH63YX2yg5WqUIu3RnDESBo.jpg
Process image : ./person/images/11/themoviedb_i_11_qiBiiT37FdqdGyd0l60pumjWYou.jpg
Process image : ./person/images/11/themoviedb_i_11_7dUkkq1lK593XvOjunlUB11lKm1.jpg
Process image : ./person/images/11/themoviedb_i_11_qDNUEcLw8g8UOaPAjbEMucB4Lc.jpg
Process image : ./person/images/11/themoviedb_i_11_y1OTC2CbAesg8mkuBbyIOc2jSKG.jpg
Process image : ./person/images/11/themoviedb_i_11_zLqk3oFN6zRMQREMUKGbamX3wC9.jpg
Process image : ./person/images/11/themoviedb_i_11_kRlx7PxXkom7Daj8Z2HmcbPQB1o.jpg
Process image : ./person/images/11/themoviedb_i_11_ueIzur9vURNLoqZCdfWltnpuZTq.jpg
Process image : ./person/images/11/themoviedb_i_11_vylDJrrtGI9fUr7W2rt0cfGIpq3.jpg
Process image : ./person/images/11/themoviedb_i_11_r5CAldfSo4t1erFc89cuAblC0CV.jpg
Process image : ./person/images/11/themoviedb_i_11_ul1ZZY0XgCZUGEPIJPMzmkz0yda.jpg
Proce

Image ./person/images/13/themoviedb_i_13_mVUxCulGluX6SYDe00Vlx5LlbBj.jpg not suitable for training: Found more than one face


Process image : ./person/images/14/themoviedb_i_14_5GilHMOt5PAQh6rlUKZzGmaKEI7.jpg
Process image : ./person/images/14/themoviedb_i_14_7nidJKw9EG21X2CnEyy9AhfRdGW.jpg
Process image : ./person/images/14/themoviedb_i_14_isQ747u0MU8U9gdsNlPngjABclH.jpg
Process image : ./person/images/14/themoviedb_i_14_z3dvKqMNDQWk3QLxzumloQVR0pv.jpg
Process image : ./person/images/14/themoviedb_i_14_th6P1UM0uzXIoE1d4AiPOumXEZw.jpg
Process image : ./person/images/14/themoviedb_i_14_zdM6RgCR5LpZwnL8UA3m7CfVpiq.jpg
Process image : ./person/images/14/themoviedb_i_14_8k1QQzdVOTwAqnZIV6eww8z9PBQ.jpg
Process image : ./person/images/14/themoviedb_i_14_vnVl4vnEb0UELwcVUGXylZ1YSb7.jpg
Process image : ./person/images/14/themoviedb_i_14_xs26A4XmOA8GT9JpRz5r7A4FXSW.jpg
Process image : ./person/images/14/themoviedb_i_14_w5yqDZo2IZV0vsrsMWUqloNiutb.jpg
Process image : ./person/images/15/themoviedb_i_15_3RdYMTLoL1X16djGF52cFtJovDT.jpg
Process image : ./person/images/15/themoviedb_i_15_7rwSXluNWZAluYMOEWBxkPmckES.jpg
Proc

...

Process image : ./person/images/98/themoviedb_i_98_adsKpz52obgGwfFCPjLsBxAHAF6.jpg
Process image : ./person/images/98/themoviedb_i_98_gIadzRlFnMb1U58M7LKIMq2YUR4.jpg
Process image : ./person/images/98/themoviedb_i_98_jAJSbtnJ38dCD0Vb3vrnRaPGQ7A.jpg
Process image : ./person/images/98/themoviedb_i_98_l4oaexOrj9KGqVmc2TK8haHfQ9E.jpg
Process image : ./person/images/98/themoviedb_i_98_csorn9i9OitOA27ThtbImrURdAS.jpg
Process image : ./person/images/98/themoviedb_i_98_jTtiPqW6VlNE9nTf4BuUzuIqzlR.jpg


Image ./person/images/98/themoviedb_i_98_csorn9i9OitOA27ThtbImrURdAS.jpg not suitable for training: Found more than one face


Process image : ./person/images/98/themoviedb_i_98_qPgSCfqFHU12KaDeYzkbq8B2uTv.jpg
Process image : ./person/images/98/themoviedb_i_98_lfW2cDigTcvuoe4k6UfCdGVE1R6.jpg
Process image : ./person/images/98/themoviedb_i_98_n1xlczx1w44DqUkJvJtAGJf9fyx.jpg
Process image : ./person/images/98/themoviedb_i_98_t1HaRL7lRJemWySXcXxOT8fAGhj.jpg


Image ./person/images/98/themoviedb_i_98_n1xlczx1w44DqUkJvJtAGJf9fyx.jpg not suitable for training: Found more than one face


Process image : ./person/images/98/themoviedb_i_98_qaO2MTBhIKcAJg2rjMmoKCIBcXF.jpg
Process image : ./person/images/99/themoviedb_i_99_23e2uoNlpDvLumNic16fS2YjZjL.jpg
Process image : ./person/images/99/themoviedb_i_99_pRfWNalyx44u98B1AjR842Ir1DF.jpg
Process image : ./person/images/99/themoviedb_i_99_onE8N8YciZtSO8hv8TBA6fRpB5b.jpg
Process image : ./person/images/99/themoviedb_i_99_sxfrOKHpm0WV72rFqGmLkzavkJS.jpg
Process image : ./person/images/99/themoviedb_i_99_4Bi7WPSr4o8pj0iOatZ9Wxdqz5h.jpg


## Load the facial recognition vectors along with the celebrity biometric data from the open web.


In [7]:
# Save model and dictonary
import pickle
trainDict = { "labels" : y , "images" : images}
pickle.dump( trainDict, open( "./models/faceLabels.pkl", "wb" ) )
pickle.dump( X,         open( "./models/faceFeatures.pkl", "wb" ) )

# Reopen to ensure content was exported correctly
faceLabels   = pickle.load( open( './models/faceLabels.pkl', 'rb' ) )
faceFeatures = pickle.load( open( './models/faceFeatures.pkl', 'rb' ) )


In [8]:
celebrityBio = pd.read_json('./models/celebrityBio.json', dtype=str)
celebrityBio.head()

,_actorID,actor,age,birthdate,gender,height
0,96,Chris Pine,39,1980-08-26,na,6' (1.83 m)
1,1,Robert Downey Jr.,54,1965-04-04,na,"5' 8½"" (1.74 m)"
2,81,Benedict Wong,48,1971-06-03,na,"5' 10"" (1.78 m)"
3,100,Taylor Lautner,27,1992-02-11,na,"5' 8½"" (1.74 m)"
4,77,Clint Eastwood,89,1930-05-31,na,"6' 4"" (1.93 m)"


In [9]:
# Append the celebrity biometric information with each facial recognition image

actorLables = pd.DataFrame( faceLabels['labels'], columns=['_actorID'])
actorLookup = actorLables.merge( celebrityBio, how='inner', left_on = '_actorID', right_on = '_actorID')
#actorLookup = actorLookup[['actorID','First Name','Last Name']]
actorNameRef =actorLookup.values.tolist()
len(actorNameRef)

pickle.dump( actorNameRef, open( "./models/actorNameRef.pkl", "wb" ) )

In [10]:
# Copy models up to S3 bucket
bucket = os.popen('aws ssm get-parameter --name jupyterBucketName-' + PLATFORM + ' --with-decryption | jq -r .Parameter.Value').read()
print(bucket)
executable = 'aws s3 sync ./models/ s3://' + bucket.strip() + '/models/ '
print(executable)
myCmd = os.popen(executable).read()
print(myCmd)

bdso-celebrity-content-dev-vpc-05447d97de18e28f8

aws s3 sync ./models/ s3://bdso-celebrity-content-dev-vpc-05447d97de18e28f8/models/ 
Completed 256.0 KiB/1.9 MiB (5.2 MiB/s) with 4 file(s) remaining
Completed 512.0 KiB/1.9 MiB (9.2 MiB/s) with 4 file(s) remaining
Completed 563.8 KiB/1.9 MiB (9.7 MiB/s) with 4 file(s) remaining
upload: models/actorNameRef.pkl to s3://bdso-celebrity-content-dev-vpc-05447d97de18e28f8/models/actorNameRef.pkl
Completed 563.8 KiB/1.9 MiB (9.7 MiB/s) with 3 file(s) remaining
Completed 819.8 KiB/1.9 MiB (13.8 MiB/s) with 3 file(s) remaining
Completed 919.7 KiB/1.9 MiB (14.7 MiB/s) with 3 file(s) remaining
upload: models/faceLabels.pkl to s3://bdso-celebrity-content-dev-vpc-05447d97de18e28f8/models/faceLabels.pkl
Completed 919.7 KiB/1.9 MiB (14.7 MiB/s) with 2 file(s) remaining
Completed 1.1 MiB/1.9 MiB (18.6 MiB/s) with 2 file(s) remaining  
Completed 1.4 MiB/1.9 MiB (21.8 MiB/s) with 2 file(s) remaining  
Completed 1.6 MiB/1.9 MiB (24.6 MiB/s) with 2 file(s)

## Elasticsearch will store image classification and perform a similary and recommendation model using the facial image vectors and biometric.

In [ ]:
# Load pre-saved models 
faceLabels = pickle.load( open( './models/faceLabels.pkl', 'rb' ) )
faceFeatures = pickle.load( open( './models/faceFeatures.pkl', 'rb' ) )
actorNameRef = pickle.load( open( './models/actorNameRef.pkl', 'rb' ) )


In [ ]:
# Preprocess of the data before loading into ES index
celebrityFacial=[]
for feature in faceFeatures:
    celebrityFacial.append(feature.tolist())
len(celebrityFacial)

celebrityImages =[]
for images in faceLabels['images']:
    image = images.split('/')
    celebrityImages.append(image[-1])
len(celebrityImages)

In [ ]:
import requests
from pprint import pprint
from datetime import datetime
from elasticsearch import Elasticsearch
import json


_index='facialimages'
esIndex = 'http://'+ESHOST+':'+ESPORT+'/'+_index  
es=Elasticsearch([{'host':ESHOST,'port':ESPORT}])

_doc= 'facialRecognition'

headers={"Content-Type": "application/json"}

## Set the index to a dense vector space to hold the 128 facial data points.

In [ ]:
# Build Elasticsearch index for facial vectors
request_body = {
  "mappings": {
    "properties": {
      "_actorID" : {
        "type" : "keyword"
      },     
      "celebrity" : {
        "type" : "keyword"
      }, 
      "facial_vector": {
        "type": "dense_vector",
        "dims": 128
      }

    }
  }
}

try:
    es.indices.delete(index = _index)
except:
    pass
    
es.indices.create(index = _index, body = request_body)

## Build the ES documents and insert for indexing

In [ ]:
# Load Facail data into ES
for i in range(len(actorNameRef)):
    _id = celebrityImages[i].split('.')[0]
    #print(_id)
    doc = {
        #_actorID	actor	age	birthdate	gender	height
        "_actorID" : actorNameRef[i][0],
        "celebrity": actorNameRef[i][1],
        "age": actorNameRef[i][2],
        "birthdate": actorNameRef[i][3],
        "gender": actorNameRef[i][4],
        "height": actorNameRef[i][5],
        "image" : celebrityImages[i],
        "facial_vector": celebrityFacial[i]  
        }
    r = requests.put(esIndex+'/_doc/'+_id, headers= headers, data = json.dumps(doc))
    print (r.text)

In [ ]:
es.indices.stats(index=_index)

## Test a given document to ensure the document was inserted correctly.

In [ ]:
r = requests.get(esIndex+'/_doc/themoviedb_i_17_mjxJd59SSdA0kiD6ju5WHpdLxy5')
pprint (r.text)

## Test Search Query based on a given facial vector space.

In [ ]:
from pprint import pprint
#celebrityFacial[1]
id=10
print ( actorNameRef[id], ' : ', celebrityImages[id])
facicalVector =[-0.0865994319319725, 0.1547723263502121, 0.11647135764360428, -0.05348784476518631, -0.026111885905265808, -0.004996640142053366, 0.02010984718799591, -0.09138417989015579, 0.16325019299983978, -0.10393447428941727, 0.26392027735710144, 0.04262522608041763, -0.17962029576301575, -0.09791891276836395, 0.09553860872983932, 0.08158333599567413, -0.29037392139434814, -0.08611540496349335, -0.05405981093645096, -0.06868094205856323, -0.05936623737215996, 0.05098782852292061, 0.08535328507423401, -0.013151603750884533, -0.059518761932849884, -0.26108041405677795, -0.07175756245851517, -0.13915595412254333, 0.15117278695106506, -0.0414716862142086, -0.06382555514574051, 0.005769657902419567, -0.2461748570203781, -0.03276572376489639, -0.062235064804553986, -0.029176823794841766, 0.03860582783818245, -0.026595083996653557, 0.19672393798828125, 0.047811634838581085, -0.09344539791345596, -0.07775542885065079, -0.0040527209639549255, 0.22311992943286896, 0.21590833365917206, -0.019895289093255997, -0.04806771129369736, 0.07343978434801102, 0.01277383416891098, -0.16670356690883636, 0.07625529915094376, 0.10308811813592911, 0.20396625995635986, 0.057876359671354294, 0.004202492069453001, -0.19998204708099365, -0.03275959938764572, -0.002369481138885021, -0.08730950951576233, 0.06798733025789261, 0.09074174612760544, -0.0927027091383934, -0.020686306059360504, -0.007380591239780188, 0.33890408277511597, 0.09122084826231003, -0.08129587769508362, -0.18075791001319885, 0.10208127647638321, -0.09331383556127548, -0.0988638624548912, 0.047538258135318756, -0.1588563323020935, -0.017572728917002678, -0.26215359568595886, 0.0427873358130455, 0.25608041882514954, 0.11964087188243866, -0.198289692401886, -0.005372968502342701, -0.20604664087295532, -0.041313473135232925, -0.00883499439805746, 0.09691508114337921, -0.09412065148353577, 0.07351617515087128, -0.07343287765979767, -0.011971116065979004, 0.12895527482032776, -0.00040591973811388016, -0.11712130159139633, 0.2198733389377594, -0.06745602190494537, 0.03355204313993454, -0.009182915091514587, -0.15589208900928497, 0.01921061798930168, -0.05102435499429703, -0.08595862984657288, -0.0336931049823761, -0.0066622281447052956, -0.12268581986427307, -0.04636511579155922, 0.1837030053138733, -0.22998324036598206, 0.0762787014245987, -0.0007653213106095791, 0.011642297729849815, 0.03597094863653183, 0.06777633726596832, -0.09956732392311096, -0.06468761712312698, 0.15399740636348724, -0.2643252909183502, 0.24139092862606049, 0.1924624890089035, 0.02700689062476158, 0.10112108290195465, 0.03542455658316612, 0.11118753254413605, -0.034934964030981064, 0.0791907086968422, -0.11021479219198227, -0.06284264475107193, 0.05167575180530548, -0.06907293945550919, 0.10757213085889816, 0.093105748295784]

q={
  "_source": {
    "includes": [ "celebrity"]
    },
  "size": 20,
  "query": {
    "script_score": {
      "query" : {
        "match_all" : {}
      },
      "script": {
        "source": "cosineSimilarity(params.query_vector, doc['facial_vector']) ", 
        "params": {
          "query_vector": facicalVector
        }
      }
    }
  }
}
res= es.search(index=_index, body=q)

pprint(res)